# Saving Numpy Arrays with Zarr

### Create Numpy Array

Let's start by creating a dummy Numpy array to work with. We'll use `np.random.rand` to generate two arrays (one small, one large) and populate both with random numbers.

In [1]:
import numpy as np

In [2]:
array_XS = np.random.rand(3,2)
array_XS

array([[0.8111477 , 0.69158708],
       [0.15792492, 0.63798994],
       [0.53240948, 0.89025667]])

In [3]:
array_L = np.random.rand(1000, 1000, 100)
array_L[:2,:2,:2]

array([[[0.79960668, 0.74690999],
        [0.06168627, 0.26227465]],

       [[0.43725292, 0.49751642],
        [0.93063213, 0.97999427]]])

### Save Numpy Array to .txt

One way to store numpy arrays is as .txt files. This works relatively well for 1- and 2-dimensional arrays, but fails for arrays in higher dimensions.

The benefit of a .txt file is that it is human-readable.

In [4]:
np.savetxt('array_XS.txt', array_XS, delimiter=" ")

In [5]:
# let's get the size (in bytes) of the stored .txt file
! stat -f '%z' array_XS.txt

150


In [6]:
np.savetxt('array_L.txt', array_L, delimiter=" ")

ValueError: Expected 1D or 2D array, got 3D array instead

### Save Numpy Array to .csv

You can use the same `np.savetxt` method to save your Numpy array to a CSV file. Make sure to set the `delimiter` keyword to ",".

Just like .txt files, .csv files are human-readable. They also have the added benefit of easy loading into DataFrames using, for example, `pd.read_csv`.

In [7]:
np.savetxt('array_XS.csv', array_XS, delimiter=",")

In [8]:
# let's get the size (in bytes) of the stored .txt file
! stat -f '%z' array_XS.csv

150


As you can see, the filesize is the same for the .txt and .csv formats.

Similar to .txt files, writing Numpy arrays to CSV only works for 1D and 2D arrays.

In [9]:
np.savetxt('array_L.csv', array_L, delimiter=",")

ValueError: Expected 1D or 2D array, got 3D array instead

You can load a numpy array stored as .CSV into a Pandas DataFrame as follows:

In [10]:
import pandas as pd
df = pd.read_csv('array_XS.csv', header=None)
df

,0,1
0,0.811148,0.691587
1,0.157925,0.637990
2,0.532409,0.890257


> NOTE: to load .txt or .csv files back into Numpy arrays correctly, make sure to use the `np.loadtxt()` method and not the `np.load()` or `np.fromfile()` methods as this may cause data-reading errors.

### Save NumPy Array with np.save()

A third way to store NumPy arrays on disk is using the native `np.save()` method which stores the arrays in binary file format.

This format allows you to save NumPy arrays in all dimensions. However, this also means it is not human-readable.

In [11]:
# save small array to binary format
np.save('array_XS.npy', array_XS)

In [12]:
# let's get the size (in bytes) of the stored .npy file
! stat -f '%z' array_XS.npy

176


In [13]:
# save large array to binary format
np.save('array_L.npy', array_L)

In [14]:
# let's get the size (in bytes) of the stored .npy file
! stat -f '%z' array_L.npy

800000128


As you can see, this file format outputs slightly larger filesizes than the previous two methods.

> NOTE: Technically there is a fourth method to store NumPy arrays, using the `np.ndarray.tofile()` method. This encode the arrays into platform-dependent binary formats however and are therefore not widely used. Read more about it [here](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.tofile.html?highlight=tofile).

## Save Numpy Array to .zarr

We've seen that the three most common ways to store numpy arrays each have their own shortcomings. TXT and CSV files can only contain 1- or 2-dimensional arrays. And the native NPY binary file format tends to lead to larger filesizes.

This is why we recommend saving your NumPy arrays with Zarr. Zarr is a format for the storage of chunked, compressed, N-dimensional arrays. It was developed as an extension and improvement on the HDF5 format.


The benefits of Zarr:
1. Can **read and write data concurrently*** in n-dimensional compressed chunks
2. Has **multiple compression options** and levels built-in
3. Is safe to use in **multiprocessing and multithreading** setups
4. Stores **metadata within the file**, allowing for flexibility 
5. Supports **multiple backend data stores** (zip, S3, etc.)
6. Has been **widely adopted** across PyData libraries like Dask, TensorStore and xarray

(*) Note that Zarr supports concurrent reads and concurrent writes separately, but not concurrent reads and writes at the same time.

Let’s see Zarr in action. Below, we’ll save the small and large arrays to .zarr and check the resulting file sizes.

In [15]:
import zarr

In [16]:
# save small numpy array to zarr
zarr.save('array_XS.zarr', array_XS)

In [17]:
# let's get the size (in bytes) of the stored .zarr file
! stat -f '%z' array_XS.zarr

128


In [18]:
# save large numpy array to zarr
zarr.save('array_L.zarr', array_L)

In [19]:
# let's get the size (in bytes) of the stored .zarr file
! du -h array_L.zarr

693M	array_L.zarr


In [74]:
693 / 800

0.86625

Storing the `array_XL` as Zarr leads to a significant reduction (~15%) in filesize, even with just the default out-of-the-box compression settings.

### Compression Options

Blosc is the default compressor used for creating Zarr arrays. You can tweak the settings of the Blosc (or any other compatible compressor) by importing it from `numcodecs` and passing it as an object class to the `compressor` keyword argument.

In [ ]:
# save large numpy array to zarr
zarr.save('array_L.zarr', array_L)

In [20]:
from numcodecs import Blosc

In [47]:
%%time
zarr_array = zarr.array(
    data=array_L, 
    chunks=True, #infers chunksize from array
    compressor=Blosc(cname="lz4hc", clevel=9), #set compression algo and level
)

CPU times: user 7.43 s, sys: 456 ms, total: 7.88 s
Wall time: 2.57 s


In [48]:
zarr.save('array_L_comp4.zarr', zarr_array)

## Load NumPy Array with Zarr

You can now load the array stored as .zarr back into your Python session using `zarr.load()`.

In [75]:
# load in array from zarr
array_zarr = zarr.load('array_M.zarr')

In [85]:
type(array_zarr)

numpy.ndarray

As you can see, when we load the .zarr file back into our Python session, it is loaded in as a regular NumPy array.

### Load Data from S3 into Zarr

In [16]:
# load small zarr array from S3
array_S = zarr.load("s3://coiled-datasets/synthetic-data/array-random-390KB.zarr")

In [20]:
array_S[:,0,0]

array([9.97862027e-01, 4.93188723e-01, 8.64042719e-01, 9.53425248e-01,
       5.92869742e-01, 1.98482100e-01, 3.78242997e-01, 9.78501028e-01,
       4.59202482e-01, 8.88982746e-01, 3.58056844e-01, 5.85341283e-01,
       7.85844688e-01, 9.11071794e-01, 5.39329780e-01, 8.61029864e-01,
       4.40726502e-01, 9.75751003e-01, 4.33597238e-01, 9.64823816e-01,
       3.31746564e-01, 2.79358177e-01, 3.08116047e-01, 8.42990623e-01,
       4.14747817e-01, 1.95971922e-01, 4.97401472e-01, 7.74970837e-01,
       6.08517834e-01, 3.06942774e-01, 6.55169935e-01, 3.26379108e-01,
       5.93332939e-01, 7.47182238e-01, 7.71864306e-01, 8.22604316e-01,
       9.17763146e-01, 9.32028668e-01, 2.58655304e-01, 9.09026001e-01,
       4.60414297e-01, 8.97946448e-01, 7.55121515e-01, 5.56243088e-01,
       6.03356205e-01, 7.66650339e-01, 9.65219838e-01, 9.90092537e-01,
       7.87905785e-01, 3.10036232e-01, 9.29806773e-01, 2.96195733e-01,
       7.15712402e-01, 8.75266389e-02, 2.43538328e-01, 9.13177378e-01,
      

In [ ]:
array_XL = zarr.load("s3://coiled-datasets/synthetic-data/array-random-370GB.zarr")

### Zarr and Dask

Due to each Zarr chunk being stored in a separate file, it is ideal for parallel access in both reading and writing. This means that Zarr and Dask work great together. Note that for efficient parallel writing, the Dask array chunks should be aligned with the Zarr target.

In [ ]:
import coiled

In [ ]:
cluster = coiled.Cluster(
    name="create-synth-array",
    software="coiled-examples/numpy-zarr",
    n_workers=50,
    shutdown_on_close=False,
    backend_options={'spot':'True'},
    scheduler_options={'idle_timeout':'2 hours'},
)

In [ ]:
from distributed import Client

In [ ]:
client = Client(cluster)
client

### TEMP: Create large Zarr array

In [1]:
import coiled

In [ ]:
# coiled.create_software_environment(
#     name="numpy-zarr",
#     account="coiled-examples",
#     conda="/Users/rpelgrim/Documents/git/coiled-resources/numpy-array-zarr/environment.yml",
# )

In [9]:
cluster = coiled.Cluster(
    name="create-synth-array",
    software="coiled-examples/numpy-zarr",
    n_workers=200,
    shutdown_on_close=False,
    backend_options={'spot':'True'},
    scheduler_options={'idle_timeout':'2 hours'},
)

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-rrpelgr71-80757-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-rrpelgr71-80757-firewall -> coiled-dask-rrpelgr71-80757-firewall]
Created FW rules: coiled-dask-rrpelgr71-80757-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-rrpelgr71-80757-cluster-firewall -> coiled-dask-rrpelgr71-80757-cluster-firewall]
Created scheduler VM: coiled-dask-rrpelgr71-80757-scheduler (type: t3.medium, ip: ['3.230.0.118'])


In [10]:
from distributed import Client

In [11]:
client = Client(cluster)
client

/Users/rpelgrim/mambaforge/envs/numpy-zarr/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+-------------+---------------+---------------+---------------+
| Package     | client        | scheduler     | workers       |
+-------------+---------------+---------------+---------------+
| dask        | 2021.11.2     | 2021.11.1     | 2021.11.1     |
| distributed | 2021.11.2     | 2021.11.1     | 2021.11.1     |
| lz4         | None          | 3.1.3         | 3.1.3         |
| msgpack     | 1.0.3         | 1.0.2         | 1.0.2         |
| numpy       | 1.21.4        | 1.21.2        | 1.21.2        |
| pandas      | 1.3.5         | 1.3.4         | 1.3.4         |
| python      | 3.9.9.final.0 | 3.9.6.final.0 | 3.9.6.final.0 |
+-------------+---------------+---------------+---------------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tls://10.4.6.144:8786' processes=164 threads=328, memory=1.21 TiB>

In [12]:
import dask.array as da

In [13]:
dask_array = da.random.random((100,100,5), chunks=(10,10,1))

In [14]:
dask_array

dask.array<random_sample, shape=(100, 100, 5), dtype=float64, chunksize=(10, 10, 1), chunktype=numpy.ndarray>

In [15]:
%%time
dask_array.to_zarr("s3://coiled-datasets/synthetic-data/array-random-390KB.zarr")

CPU times: user 3.7 s, sys: 417 ms, total: 4.12 s
Wall time: 25 s


In [19]:
import zarr

### Chunking

Not totally sure yet how to demonstrate Zarr chunking.

In [7]:
import zarr
import numpy as np

In [3]:
z = zarr.zeros((10000, 10000), chunks=(1000, 1000), dtype='i4')
z

<zarr.core.Array (10000, 10000) int32>

In [4]:
# write scalar value to entire array
z[:] = 42

In [8]:
# write scalar values to specific regions of the array
z[0, :] = np.arange(10000)
z[:, 0] = np.arange(10000)

In [11]:
# retrieve contents by slicing
z[0, :10], z[-1,-1]

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32), 42)